In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install rouge
import json
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from rouge import Rouge
import nltk.translate
from nltk.translate import meteor_score
import progressbar
import numpy as np
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
path_gold = '/content/drive/MyDrive/Thesis/Models_whole_data/case_explanation/metrics and results/gold_explanations_ranked.json'
path_occ = '/content/drive/MyDrive/Thesis/Models_whole_data/case_explanation/LSGBERT_bigru/occlusion_scores/occ_exp_full_0.4.json'
g_json = open(path_gold, "r")
gold_exp = json.load(g_json)
o_json = open(path_occ, "r")
occ_exp = json.load(o_json)

In [ ]:
files = list(occ_exp.keys())

In [ ]:
print(files)

['1962_213.txt', '1962_47.txt', '1951_35.txt', '1953_74.txt', '1960_100.txt', '1960_72.txt', '1963_37.txt', '1951_33.txt', '1952_42.txt', '1953_26.txt', '1962_128.txt', '1959_5.txt', '1951_30.txt', '1962_118.txt', '1952_60.txt', '1951_40.txt', '1959_26.txt', '1961_363.txt', '1954_144.txt', '1951_64.txt', '1960_12.txt', '1954_0.txt', '1961_344.txt', '1959_76.txt', '1960_327.txt', '1951_36.txt', '1954_114.txt', '1959_66.txt', '1960_10.txt', '1962_105.txt', '1961_365.txt', '1962_207.txt', '1962_339.txt', '1960_103.txt', '1961_400.txt', '1952_75.txt', '1953_57.txt', '1961_417.txt', '1959_189.txt', '1959_134.txt', '1960_87.txt', '1962_384.txt', '1954_13.txt', '1960_44.txt', '1951_80.txt', '1951_10.txt', '1954_158.txt', '1953_14.txt', '1962_113.txt', '1960_265.txt', '2013_35.txt', '2003_794.txt', '1999_1001.txt', '2013_30.txt', '2013_95.txt', '2013_57.txt']


In [ ]:
def get_BLEU_score(ref_text, machine_text):
    tok_ref_text = word_tokenize(ref_text)
    tok_machine_text = word_tokenize(machine_text)
    score = nltk.translate.bleu_score.sentence_bleu([tok_ref_text], tok_machine_text, weights = (0.5,0.5))
    return score

In [ ]:
def get_METEOR_score(ref_text, machine_text):
    tok_ref_text = word_tokenize(ref_text)
    tok_machine_text = word_tokenize(machine_text)
    score = nltk.translate.meteor_score.meteor_score([tok_ref_text], tok_machine_text)
    return score

In [ ]:
def jaccard_similarity(query, document):
    query = word_tokenize(query)
    document = word_tokenize(document)
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    if(len(union)==0):
        return 0
    return len(intersection)/len(union)

In [ ]:
def overlap_coefficient_min(query, document):
    query = word_tokenize(query)
    document = word_tokenize(document)
    intersection = set(query).intersection(set(document))
    den = min(len(set(query)),len(set(document)))
    if(den==0):
        return 0
    return len(intersection)/den

In [ ]:
def overlap_coefficient_max(query, document):
    query = word_tokenize(query)
    document = word_tokenize(document)
    intersection = set(query).intersection(set(document))
    den = max(len(set(query)),len(set(document)))
    if(den==0):
        return 0
    return len(intersection)/den

In [ ]:
def occ_result_maker(file_to_write, Rank_initial, Rank_final, occ_exp, gold_exp):
    rouge1 = []
    rouge2 = []
    rougel = []
    jaccard = []
    bleu = []
    meteor = []
    overlap_min = []
    overlap_max = []

    for u in range(5):
        user = "User " + str(u+1)
        r1 = []
        r2 = []
        rl = []
        jacc = []
        bl = []
        met = []
        omin = []
        omax = []

        for i in progressbar.progressbar(range(len(files))):
            f = files[i]
            ref_text = ""
            for rank in range(Rank_initial, Rank_final+1, 1):
                if(gold_exp[f][user]["exp"]["Rank" + str(rank)]!=""): # check if user x has sentence in rank y
                    ref_text += gold_exp[f][user]["exp"]["Rank" + str(rank)] + " " # if yes save the text of all the renk considered in ref_text

            machine_text = occ_exp[f]
            machine_text = machine_text.lower()
            ref_text = ref_text.lower()

            if(ref_text == ""):
                continue
            rouge = Rouge()
            score = rouge.get_scores(machine_text, ref_text)
            r1.append(score[0]['rouge-1']['f'])
            r2.append(score[0]['rouge-2']['f'])
            rl.append(score[0]['rouge-l']['f'])
            jacc.append(jaccard_similarity(ref_text, machine_text))
            omin.append(overlap_coefficient_min(ref_text, machine_text))
            omax.append(overlap_coefficient_max(ref_text, machine_text))
            bl.append(get_BLEU_score(ref_text, machine_text))
            met.append(get_METEOR_score(ref_text, machine_text))

        rouge1.append(np.mean(r1))
        rouge2.append(np.mean(r2))
        rougel.append(np.mean(rl))
        jaccard.append(np.mean(jacc))
        overlap_min.append(np.mean(omin))
        overlap_max.append(np.mean(omax))
        bleu.append(np.mean(bl))
        meteor.append(np.mean(met))

    file_to_write.write("ROUGE-1 : {:}".format(rouge1) + "\n\n")
    file_to_write.write("ROUGE-2 : {:}".format(rouge2) + "\n\n")
    file_to_write.write("ROUGE-L : {:}".format(rougel)+ "\n\n")
    file_to_write.write("Jaccard : {:}".format(jaccard)+ "\n\n")
    file_to_write.write("Overmin : {:}".format(overlap_min)+ "\n\n")
    file_to_write.write("Overmax : {:}".format(overlap_max)+ "\n\n")
    file_to_write.write("BLEU    : {:}".format(bleu)+ "\n\n")
    file_to_write.write("METEOR  : {:}".format(meteor)+ "\n\n")

In [ ]:
experiments = [(1,10), (1,1), (2,2), (3,3), (4,4), (5,5), (6,6), (7,7), (8,8), (9,9), (10,10), (1,5), (5,10)]
for exp in experiments:
    print(exp)
    f = open("/content/drive/MyDrive/Thesis/Models_whole_data/case_explanation/metrics and results/result_files/Rank_" + str(exp[0]) + "_to_" + str(exp[1]) + ".txt", "w")
    occ_result_maker(f, exp[0], exp[1], occ_exp, gold_exp)

  0% (0 of 56) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

(1, 10)


100% (56 of 56) |########################| Elapsed Time: 0:01:32 Time:  0:01:32
100% (56 of 56) |########################| Elapsed Time: 0:00:52 Time:  0:00:52
100% (56 of 56) |########################| Elapsed Time: 0:01:54 Time:  0:01:54
100% (56 of 56) |########################| Elapsed Time: 0:02:12 Time:  0:02:12
100% (56 of 56) |########################| Elapsed Time: 0:01:08 Time:  0:01:08
  0% (0 of 56) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

(1, 1)


100% (56 of 56) |########################| Elapsed Time: 0:00:12 Time:  0:00:12
100% (56 of 56) |########################| Elapsed Time: 0:00:14 Time:  0:00:14
100% (56 of 56) |########################| Elapsed Time: 0:00:47 Time:  0:00:47
100% (56 of 56) |########################| Elapsed Time: 0:00:10 Time:  0:00:10
100% (56 of 56) |########################| Elapsed Time: 0:00:10 Time:  0:00:10
  0% (0 of 56) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

...

100% (56 of 56) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (56 of 56) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
  0% (0 of 56) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

(1, 5)


100% (56 of 56) |########################| Elapsed Time: 0:01:17 Time:  0:01:17
100% (56 of 56) |########################| Elapsed Time: 0:00:55 Time:  0:00:55
100% (56 of 56) |########################| Elapsed Time: 0:01:56 Time:  0:01:56
100% (56 of 56) |########################| Elapsed Time: 0:02:09 Time:  0:02:09
100% (56 of 56) |########################| Elapsed Time: 0:00:53 Time:  0:00:53
  0% (0 of 56) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

(5, 10)


100% (56 of 56) |########################| Elapsed Time: 0:00:07 Time:  0:00:07
100% (56 of 56) |########################| Elapsed Time: 0:00:02 Time:  0:00:02
100% (56 of 56) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (56 of 56) |########################| Elapsed Time: 0:00:16 Time:  0:00:16
100% (56 of 56) |########################| Elapsed Time: 0:00:27 Time:  0:00:27


In [ ]:
# COMPUTE MEAN TOKENS IN ANNOTED DATASET

In [ ]:
import pandas as pd
# load the entire test set
dataset = pd.read_csv('/content/drive/MyDrive/Thesis/train_data/ILDC_multi.csv')
test = dataset.loc[dataset['split'] == 'test']

In [ ]:
# filtered test set
test = test.reset_index(drop=True) # reset index of the test set
filter = test['name'].isin(files) # create a filter that includes only the docs contained in ILDCexpert
df = test[filter] # dataframe of the documents in ILDCexpert -> lenght = 56

In [ ]:
tot=0
for i in range(len(df)):
  text = df.iloc[i]['text']
  tok_text = word_tokenize(text)
  tot += len(tok_text)

print(f'The mean tokens of the annotated docs is: {tot/len(df)}')

The mean tokens of the annotated docs is: 3162.214285714286


In [ ]:
tot = 0
for i in range(len(files)):
  f = files[i]
  machine_text = occ_exp[f]
  tok_machine_text = word_tokenize(machine_text)
  tot += len(tok_machine_text)

print(f'The mean tokens of the machine docs is: {tot/56}')

The mean tokens of the machine docs is: 717.75


In [ ]:
path_authors = '/content/drive/MyDrive/Thesis/Models_whole_data/case_explanation/metrics and results/occ_explanations.json'
a_json = open(path_authors, "r")
occ_authors = json.load(a_json)

In [ ]:
tot = 0
for i in range(len(files)):
  f = files[i]
  machine_text = occ_authors[f]
  tok_machine_text = word_tokenize(machine_text)
  tot += len(tok_machine_text)

print(f'The mean tokens of the machine docs of the AUTHORS is: {tot/56}')

The mean tokens of the machine docs of the AUTHORS is: 730.4107142857143
